## Autor:Diego Amoroso
## Modelo:bert-base-multilingual-uncased



In [1]:
#Instalamos transformers

!pip install transformers

#Instalamos PyDrive
!pip install -U -q PyDrive

In [ ]:
#Instalamos PyDrive wrapper e importamos las librerias.

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user() #Autenticar usuario google
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '#' #ID del archivo
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
# Descargamos en archivo en el disco local como '.xlsx'.

downloaded.GetContentFile('DF-testmodels.xlsx')

In [ ]:
# Ubicacion del archivo en el disco

!ls -lha DF-testmodels.xlsx

In [ ]:
# Usamos pandas read_excel despues de instalar la libreria para importar excel.
!pip install -q xlrd

import pandas as pd
df = pd.read_excel('DF-testmodels.xlsx')


lista_codaudio = list(df['ARCHIVO'])
lista_texto = list(df['TEXTO'])
print(lista_codaudio)
print(lista_texto)



In [ ]:
#------------------------------FUNCIONES------------------------------


#-------------------Funcion Convertir String---------------------
def listToString(s):
    str1 = " "

    for ele in s:
        str1 = str1 + " " + ele

    return str1


# Retorna una Lista con los codigos
def GenId(v,num):
  a = []
  n = num
  for i in range(len(v)):
    pal=""
    if n < 10:
      pal = "da-0" + str(n)
      n += 1
      a.append(pal)
    else:
      pal = "da-" + str(n)
      n += 1
      a.append(pal)

  return a




## Retorna una Lista con el Nombre
def GenOwner(v):
  a = []
  for i in range(len(v)):
    a.append("diego amoroso")

  return a



## Retorna una Lista con el Modelo
def GenModel(v):
  a = []
  b = []
  for i in range(len(v)):
    a.append("bert-base-multilingual-uncased ")

  return a


## Retorna una Lista con el La Version
def GenVersion(v):
  a = []
  for i in range(len(v)):
    a.append("1")

  return a



## Retorna una Lista con word_Match
def GenWordMatch(v,v2):
  a = []
  for i in range(len(v2)):
    if v[i] == v2[i]:
      a.append("1")
    else:
      a.append("0")
  return a



## Retorna una Lista con codigo del audio
def GenCodAudio(posaudio,v,v2):#(posicion del audio, vector texto, vector audios)
  a = []
  for i in range(len(v)):
    a.append(v2[posaudio])
  return a

## Retorna una Lista con la palabra
def GenWord(palabra,v):
  a = []
  for i in range(len(v)):
    a.append(palabra)
  return a

  ## Retorna una Lista con la cantidad de [MASK]
def GenCantMask(v,v2):
  a = []
  for i in range(len(v)):
    a.append(len(v2))
  return a


## Retorna una Lista con la posicion de la palabra
def GenPosPal(j,v):
  a = []

  for i in range(len(v)):
    a.append(str(j+1))
  return a





In [ ]:
from transformers import pipeline

#Creacion Dataframe Vacio
import pandas as pd

#importar libreria Time
import time

df = pd.DataFrame(columns=['id', 'owner','model','version','cod_audio','word','mask_qua','mask_num','word_pos','token','token_str','token_score','word_match'])



i=0
numero=1 # numero para el codigo GenId()

for i in range(len(lista_texto)): #itera en un vector con todos los textos
  txt = lista_texto[i]
  vec_texto = txt.split()


  unmasker = pipeline('fill-mask', model='bert-base-multilingual-uncased')

  for j in range(len(vec_texto)): #itera en cada palabra del texto

    start_time = time.time()

    print(f"Audio: { i }\n")
    vec_texto = txt.split()
    palabra = vec_texto[j]

    vec_txtmask = vec_texto
    vec_txtmask[j] = "[MASK]"
    txt_mask = listToString(vec_txtmask)

    print(f"Texto Enmascarado:{txt_mask}\n")
    lista_unmasked = unmasker(txt_mask)

    lista_token = []
    lista_token_str = []
    lista_token_score = []

    for k in range(len(lista_unmasked)):
      diccionario_unmasked = lista_unmasked[k]

      lista_token.append(diccionario_unmasked["token"])
      lista_token_str.append(diccionario_unmasked["token_str"])
      lista_token_score.append(diccionario_unmasked["score"])


    print("-----------------------------------------------------------------")
    print(f"\|Codigos: {GenId(lista_unmasked,numero)}")
    print(f"\|Nombres: {GenOwner(lista_unmasked)}")
    print(f"\|Modelos: {GenModel(lista_unmasked)}")
    print(f"\|Version: {GenVersion(lista_unmasked)}")
    print(f"\|Codig Audio: {GenCodAudio(i,lista_unmasked,lista_codaudio)}")
    print(f"\|word : {palabra} - {GenWord(palabra,lista_unmasked)}")
    print(f"\|Word match: {GenWordMatch(GenWord(palabra,lista_unmasked),lista_token_str)}")
    print(f"\|cantidad mask : {len(vec_texto)} - {GenCantMask(lista_unmasked,vec_texto)}")
    print(f"\|Posicion pal: { j } / {len(vec_texto)} - {GenPosPal(j,lista_unmasked)}")
    print(f"\|token : {lista_token}")
    print(f"\|token: str {lista_token_str}")
    print(f"\|token_score {lista_token_score} ")
    print("-----------------------------------------------------------------")
    print("--- %s segundos ---" % (time.time() - start_time))
    print("\n")
    print("\n")

    #Agregar datos al dataframe
    id = GenId(lista_unmasked,numero)
    owner = GenOwner(lista_unmasked)
    model = GenModel(lista_unmasked)
    version = GenVersion(lista_unmasked)
    cod_audio = GenCodAudio(i,lista_unmasked,lista_codaudio)
    word = GenWord(palabra,lista_unmasked)
    mask_qua = GenCantMask(lista_unmasked,vec_texto)
    mask_num = GenPosPal(j,lista_unmasked)
    word_pos = GenPosPal(j,lista_unmasked)
    token = lista_token
    token_str = lista_token_str
    token_score = lista_token_score
    word_match = GenWordMatch(GenWord(palabra,lista_unmasked),lista_token_str)

    for it in range(len(id)):
      dict_datos = {
        "id" : id[it],
        "owner" : owner[it],
        "model" : model[it],
        "version" : version[it],
        "cod_audio" : cod_audio[it],
        "word" : word[it],
        "mask_qua" : mask_qua[it],
        "mask_num" : mask_num[it],
        "word_pos" : word_pos[it],
        "token" : token[it],
        "token_str" : token_str[it],
        "token_score" : token_score[it],
        "word_match" : word_match[it],
      }
      numero += 1
      df = df.append(dict_datos,ignore_index=True)





In [ ]:
#Exportar a excel

df.to_excel('Bert-Model-Table.xlsx', index=False)

In [ ]:
#Ubicacion archivo exportado en el Disco

!ls -lha Bert-Model-Table.xlsx

In [ ]:
#Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#Descarga en la maquina

from google.colab import files
files.download('Bert-Model-Table.xlsx')